In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

# 1

In [2]:
df = pd.read_csv('df_text_preprocessing.csv')

In [3]:
df = df.drop(['Unnamed: 0'],axis = 'columns')

In [4]:
df.head(1)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"['what', 'is', 'the', 'step', 'by', 'step', 'g...","['what', 'is', 'the', 'step', 'by', 'step', 'g...",0


# 2

In [5]:
df1 = pd.read_csv('df.csv')

In [6]:
df1 = df1.drop(['Unnamed: 0','qid1','qid2','question1','question2','is_duplicate'],axis = 'columns')

In [7]:
df1.head(1)

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_common,word_total,word_share,freq_q1+freq_q2,freq_q1-freq_q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0


# 3

In [8]:
df2 = pd.read_csv('df_after_cleaning.csv')

In [9]:
df2 = df2.drop(['Unnamed: 0','qid1','qid2','question1','question2','is_duplicate'],axis = 'columns')

In [10]:
df2.head(1)

,id,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0.99998,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759


### Merge all datasets in one 

In [11]:
df = df.merge(df1,on='id',how='left')

In [12]:
df.head(1)

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_common,word_total,word_share,freq_q1+freq_q2,freq_q1-freq_q2
0,0,1,2,"['what', 'is', 'the', 'step', 'by', 'step', 'g...","['what', 'is', 'the', 'step', 'by', 'step', 'g...",0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0


In [13]:
df = df.merge(df2,on='id',how='left')

In [14]:
df.head(1)

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,1,2,"['what', 'is', 'the', 'step', 'by', 'step', 'g...","['what', 'is', 'the', 'step', 'by', 'step', 'g...",0,1,1,66,57,...,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759


In [15]:
df.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate',
       'freq_qid1', 'freq_qid2', 'q1len', 'q2len', 'q1_n_words', 'q2_n_words',
       'word_common', 'word_total', 'word_share', 'freq_q1+freq_q2',
       'freq_q1-freq_q2', 'cwc_min', 'cwc_max', 'csc_min', 'csc_max',
       'ctc_min', 'ctc_max', 'last_word_eq', 'first_word_eq', 'abs_len_diff',
       'mean_len', 'token_set_ratio', 'token_sort_ratio', 'fuzz_ratio',
       'fuzz_partial_ratio', 'longest_substr_ratio'],
      dtype='object')

In [16]:
df.shape

(404290, 32)

In [17]:
df

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,1,2,"['what', 'is', 'the', 'step', 'by', 'step', 'g...","['what', 'is', 'the', 'step', 'by', 'step', 'g...",0,1,1,66,57,...,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,3,4,"['what', 'is', 'the', 'story', 'of', 'kohinoor...","['what', 'would', 'happen', 'if', 'the', 'indi...",0,4,1,51,88,...,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,5,6,"['how', 'can', 'i', 'increase', 'the', 'speed'...","['how', 'can', 'internet', 'speed', 'be', 'inc...",0,1,1,73,59,...,0.285712,0.0,1.0,4.0,12.0,63,63,43,47,0.166667
3,3,7,8,"['why', 'am', 'i', 'mentally', 'very', 'lonely...","['find', 'the', 'remainder', 'when', 'math', '...",0,1,1,50,65,...,0.000000,0.0,0.0,2.0,12.0,28,24,9,14,0.039216
4,4,9,10,"['which', 'one', 'dissolve', 'in', 'water', 'q...","['which', 'fish', 'would', 'survive', 'in', 's...",0,3,1,76,39,...,0.307690,0.0,1.0,6.0,10.0,67,47,35,56,0.175000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404285,404285,433578,379845,"['how', 'many', 'keywords', 'are', 'there', 'i...","['how', 'many', 'keywords', 'are', 'there', 'i...",0,2,2,85,79,...,0.785709,1.0,1.0,1.0,13.5,97,88,91,87,0.387500
404286,404286,18840,155606,"['do', 'you', 'believe', 'there', 'is', 'life'...","['is', 'it', 'true', 'that', 'there', 'is', 'l...",1,12,1,41,42,...,0.555549,1.0,0.0,1.0,8.5,79,69,72,76,0.642857
404287,404287,537928,537929,"['what', 'is', 'one', 'coin']","['what', 'is', 'this', 'coin']",0,1,1,17,17,...,0.749981,1.0,1.0,0.0,4.0,86,79,79,76,0.444444
404288,404288,537930,537931,"['what', 'is', 'the', 'approx', 'annual', 'cos...","['i', 'am', 'having', 'little', 'hairfall', 'p...",0,1,1,94,127,...,0.040000,0.0,0.0,8.0,21.0,37,35,30,34,0.052632


# TF-IDF

<img src="tfidf.jpg" width="600" height="300">

### Here i have created a small blog explainig Term Frequency Inverse Document Frequency 

https://niharjamdar.medium.com/tf-idf-term-frequency-and-inverse-document-frequency-56a0289d2fb6

<img src="logasidf1.jpg" width="600" height="300">

Note : 
       TF gives more importance to word (wi) which occurs more often in sentence (sj) so that Term Freq will increase
       
    IDF gives more importance to word (wi) which occurs very rare in whole corpus so that IDF will increase

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [34]:
import spacy 

spacy.cli.download("en_core_web_sm")

nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [46]:
# en_vectors_web_lg, which includes over 1 million unique vectors.


vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)

100%|████████████████████████████████| 404290/404290 [5:45:51<00:00, 19.48it/s]


In [48]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|███████████████████████████████| 404290/404290 [13:24:06<00:00,  8.38it/s]


In [50]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,...,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,q1_feats_m,q2_feats_m
0,0,1,2,"['what', 'is', 'the', 'step', 'by', 'step', 'g...","['what', 'is', 'the', 'step', 'by', 'step', 'g...",0,1,1,66,57,...,1.0,2.0,13.0,100,93,93,100,0.982759,"[-14.745891690254211, 24.718946654349566, 32.0...","[-14.794604361057281, 21.084941882640123, 28.1..."
1,1,3,4,"['what', 'is', 'the', 'story', 'of', 'kohinoor...","['what', 'would', 'happen', 'if', 'the', 'indi...",0,4,1,51,88,...,1.0,5.0,12.5,86,63,66,75,0.596154,"[-41.812612533569336, 44.350191459059715, 15.2...","[-58.999562591314316, 50.63228598237038, 24.49..."
2,2,5,6,"['how', 'can', 'i', 'increase', 'the', 'speed'...","['how', 'can', 'internet', 'speed', 'be', 'inc...",0,1,1,73,59,...,1.0,4.0,12.0,63,63,43,47,0.166667,"[3.016371876001358, 30.276006504893303, 11.130...","[3.447697162628174, 18.027316480875015, 17.648..."
3,3,7,8,"['why', 'am', 'i', 'mentally', 'very', 'lonely...","['find', 'the', 'remainder', 'when', 'math', '...",0,1,1,50,65,...,0.0,2.0,12.0,28,24,9,14,0.039216,"[-40.13401734828949, -9.230085268616676, 6.283...","[10.574279189109802, 54.67509198188782, 34.278..."
4,4,9,10,"['which', 'one', 'dissolve', 'in', 'water', 'q...","['which', 'fish', 'would', 'survive', 'in', 's...",0,3,1,76,39,...,1.0,6.0,10.0,67,47,35,56,0.175000,"[-31.898521184921265, 65.72667498886585, 28.69...","[-18.031274497509003, 33.73362094536424, 9.502..."


In [51]:
df3_q1 = pd.DataFrame(df.q1_feats_m.values.tolist(), index= df.index)

In [52]:
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,-14.745892,24.718947,32.092726,-29.658814,-14.809326,39.054106,-43.936551,-30.712149,10.277332,-21.445892,...,5.291891,-34.579369,-60.617696,-61.315548,9.161167,15.889777,20.078552,-32.007888,-43.683638,-7.127860
1,-41.812613,44.350191,15.261093,-24.972682,-16.441374,28.734205,-72.027922,-22.179564,19.328411,-36.784459,...,-0.157103,-24.639804,-47.139824,-65.808416,1.492900,34.403210,54.401606,-22.481677,-27.236494,-8.253206
2,3.016372,30.276007,11.130095,-22.573581,-33.075505,23.782594,-34.367002,-20.302041,6.471570,-29.689112,...,1.702879,-21.262129,-34.186675,-53.481558,10.512593,32.417519,16.473425,4.559537,-27.917456,-27.839384
3,-40.134017,-9.230085,6.283272,-12.877799,-16.168017,16.644989,-36.557493,-38.394760,4.331502,-28.671539,...,-14.992825,-8.113776,-19.178638,-17.569917,-12.641262,3.808732,11.670762,-5.199494,8.149668,-11.443322
4,-31.898521,65.726675,28.690825,-51.455148,-47.701419,36.724667,-51.505395,-42.829610,-5.379189,-32.632293,...,23.084680,-48.643830,-91.722090,-62.073409,20.977263,37.945637,67.437372,-34.511794,-56.211583,-25.387325


In [53]:
df3_q2 = pd.DataFrame(df.q2_feats_m.values.tolist(), index= df.index)

In [54]:
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,-14.794604,21.084942,28.175086,-24.669877,-9.605722,34.278758,-39.162006,-25.157176,9.675985,-16.250439,...,5.613818,-32.621258,-55.788514,-54.075668,3.277048,12.701243,14.444472,-28.278374,-41.664311,-10.190590
1,-58.999563,50.632286,24.493231,-33.410660,-22.316718,39.254143,-94.014250,-37.877661,13.548109,-46.971156,...,-6.675118,-38.421278,-63.571089,-83.859878,3.667343,35.065960,66.988533,-35.691742,-34.204698,-25.940935
2,3.447697,18.027316,17.648600,-5.228206,-18.170197,38.857863,-41.307360,-16.616749,10.830895,-16.358831,...,-32.653073,-26.145077,-27.134681,-49.700357,22.640742,26.780651,38.932618,-29.056358,-14.098857,-25.258516
3,10.574279,54.675092,34.278645,-46.804962,21.825762,7.099904,-55.927039,-21.548024,16.181055,-25.945377,...,-26.034915,-49.547621,-42.007318,-71.782417,21.370941,32.258106,41.693433,-39.583549,-28.672596,19.400807
4,-18.031274,33.733621,9.502852,-13.156332,-12.256794,27.968353,-12.777182,-17.911293,1.725892,-18.089952,...,9.280396,-15.230804,-37.091534,-35.460836,2.832767,6.925419,22.764415,-20.300345,-13.979813,-15.314046


# So How many Features do we have for training ?

### Orignal features + 11 features before extraction 15 Advanced features after extraction + 96 Tf-idfWeightedfeatures of Question1  + 96 Tf-idfWeightedfeatures of Question2

In [55]:
df.head(2)

,id,qid1,qid2,question1,question2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,...,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio,q1_feats_m,q2_feats_m
0,0,1,2,"['what', 'is', 'the', 'step', 'by', 'step', 'g...","['what', 'is', 'the', 'step', 'by', 'step', 'g...",0,1,1,66,57,...,1.0,2.0,13.0,100,93,93,100,0.982759,"[-14.745891690254211, 24.718946654349566, 32.0...","[-14.794604361057281, 21.084941882640123, 28.1..."
1,1,3,4,"['what', 'is', 'the', 'story', 'of', 'kohinoor...","['what', 'would', 'happen', 'if', 'the', 'indi...",0,4,1,51,88,...,1.0,5.0,12.5,86,63,66,75,0.596154,"[-41.812612533569336, 44.350191459059715, 15.2...","[-58.999562591314316, 50.63228598237038, 24.49..."


In [56]:
df1 = df.drop(['question1','question2','q1_feats_m','q2_feats_m'],axis = 'columns')

In [57]:
df1.head()

,id,qid1,qid2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,1,2,0,1,1,66,57,14,12,...,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,3,4,0,4,1,51,88,8,13,...,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,5,6,0,1,1,73,59,14,10,...,0.285712,0.0,1.0,4.0,12.0,63,63,43,47,0.166667
3,3,7,8,0,1,1,50,65,11,9,...,0.000000,0.0,0.0,2.0,12.0,28,24,9,14,0.039216
4,4,9,10,0,3,1,76,39,13,7,...,0.307690,0.0,1.0,6.0,10.0,67,47,35,56,0.175000


In [58]:
df3_q1['id'] = df1['id']
df3_q2['id'] = df1['id']

In [60]:
df1 = df1.merge(df3_q1,on='id',how='left')

In [61]:
df1.head(2)

,id,qid1,qid2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,...,86,87,88,89,90,91,92,93,94,95
0,0,1,2,0,1,1,66,57,14,12,...,5.291891,-34.579369,-60.617696,-61.315548,9.161167,15.889777,20.078552,-32.007888,-43.683638,-7.127860
1,1,3,4,0,4,1,51,88,8,13,...,-0.157103,-24.639804,-47.139824,-65.808416,1.492900,34.403210,54.401606,-22.481677,-27.236494,-8.253206


In [62]:
df1 = df1.merge(df3_q2,on='id',how='left')

In [63]:
df1.head(2)

,id,qid1,qid2,is_duplicate,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,...,86_y,87_y,88_y,89_y,90_y,91_y,92_y,93_y,94_y,95_y
0,0,1,2,0,1,1,66,57,14,12,...,5.613818,-32.621258,-55.788514,-54.075668,3.277048,12.701243,14.444472,-28.278374,-41.664311,-10.190590
1,1,3,4,0,4,1,51,88,8,13,...,-6.675118,-38.421278,-63.571089,-83.859878,3.667343,35.065960,66.988533,-35.691742,-34.204698,-25.940935


In [69]:
df1.shape

(404290, 222)

In [70]:
df1.to_csv('finalized.csv',index = 'False')